In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np
import CifarData

CIFAR_DIR = "./../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
"""
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
with 6000 images per class. There are 50000 training images and 10000 test images. 

## data -- a 10000x3072(32*32*3) numpy array of uint8s. 
Each row of the array stores a 32x32 colour image. 
The first 1024 entries contain the red channel values, 
the next 1024 the green, and the final 1024 the blue. 
The image is stored in row-major order, so that the first 32 entries of the array 
are the red channel values of the first row of the image.

## labels -- a list of 10000 numbers in the range 0-9. 
The number at index i indicates the label of the ith image in the array data.
"""

'\n#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, \nwith 6000 images per class. There are 50000 training images and 10000 test images. \n\n## data -- a 10000x3072(32*32*3) numpy array of uint8s. \nEach row of the array stores a 32x32 colour image. \nThe first 1024 entries contain the red channel values, \nthe next 1024 the green, and the final 1024 the blue. \nThe image is stored in row-major order, so that the first 32 entries of the array \nare the red channel values of the first row of the image.\n\n## labels -- a list of 10000 numbers in the range 0-9. \nThe number at index i indicates the label of the ith image in the array data.\n'

In [3]:
"""
testing 
"""
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData.CifarData(train_filenames, True)
test_data = CifarData.CifarData(test_filenames, False)

tst_batch_data, tst_batch_labels = train_data.next_batch(10)
print(tst_batch_data)
print(tst_batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[ 0.92941176  0.94509804  0.95294118 ...,  0.22352941  0.15294118
   0.1372549 ]
 [ 0.4745098   0.42745098  0.46666667 ...,  0.00392157  0.01176471
   0.01960784]
 [ 0.73333333  0.71764706  0.7254902  ..., -0.38823529 -0.38823529
  -0.35686275]
 ..., 
 [ 0.1372549   0.09019608  0.09803922 ...,  0.05882353  0.03529412
   0.01176471]
 [ 0.21568627  0.24705882  0.28627451 ...,  0.03529412  0.01960784
   0.01176471]
 [-0.40392157 -0.42745098 -0.54509804 ..., -0.41960784 -0.28627451
  -0.33333333]]
[8 0 9 3 1 4 0 8 6 6]


In [4]:
"""
create compute graph
single layer net work with 10 neurons
"""
x = tf.placeholder(tf.float32, [None, 3072])
# [None], eg: [0,5,6,3]
y = tf.placeholder(tf.int64, [None])

# 10-classification need 10 outputs
# (3072, 10)
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                   initializer=tf.random_normal_initializer(0, 1))
# (10, )
b = tf.get_variable('b', [10],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b

# mean square loss
"""
# course: 1 + e^x
# api: e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], [], ...] # 10000*[], 
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

# cross entropy loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# what does it do:
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_



# indices
predict = tf.argmax(y_, 1)  # dim 0 or dim 1   
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [6]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# run 100k: 30.95%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
                                        [loss, accuracy, train_op],
                                        feed_dict={
                                            x: batch_data,
                                            y: batch_labels}
                                        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData.CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                                        [accuracy],
                                        feed_dict = {
                                            x: test_batch_data, 
                                            y: test_batch_labels}
                                        )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
                

[Train] Step: 500, loss: 24.65310, acc: 0.10000
[Train] Step: 1000, loss: 15.06285, acc: 0.20000
[Train] Step: 1500, loss: 9.73833, acc: 0.15000
[Train] Step: 2000, loss: 15.42166, acc: 0.25000
[Train] Step: 2500, loss: 7.89101, acc: 0.30000
[Train] Step: 3000, loss: 7.93487, acc: 0.30000
[Train] Step: 3500, loss: 11.79579, acc: 0.15000
[Train] Step: 4000, loss: 8.43897, acc: 0.25000
[Train] Step: 4500, loss: 5.28395, acc: 0.25000
[Train] Step: 5000, loss: 7.86700, acc: 0.25000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.24400
[Train] Step: 5500, loss: 11.23963, acc: 0.30000
[Train] Step: 6000, loss: 6.48047, acc: 0.40000
[Train] Step: 6500, loss: 3.88700, acc: 0.35000
[Train] Step: 7000, loss: 5.99141, acc: 0.30000
[Train] Step: 7500, loss: 7.26124, acc: 0.20000
[Train] Step: 8000, loss: 4.84996, acc: 0.25000
[Train] Step: 8500, loss: 7.09732, acc: 0.35000
[Train] Step: 9000, loss: 6.87352, acc: 0.20000
[Train] Step: 9500, loss: 6.81222, acc: 0.35000
[Train] Step: 10000, loss: 4